Variational Autoencoder with 1D convolutional layer

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
# Keras uses TensforFlow backend as default
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

# Input image dimensions
steps, original_dim = 1, 28*28 # Take care here since we are changing this according to the data
# Number of convolutional filters to use
filters = 64
# Convolution kernel size
num_conv = 6
# Set batch size
batch_size = 100
# Decoder output dimensionality
decOutput = 10

latent_dim = 20
intermediate_dim = 256
epsilon_std = 1.0
epochs = 5

x = Input(batch_shape=(batch_size,steps,original_dim))
# Play around with padding here, not sure what to go with.
conv_1 = Conv1D(1,
                kernel_size=num_conv,
                padding='same', 
                activation='relu')(x)
conv_2 = Conv1D(filters,
                kernel_size=num_conv,
                padding='same', 
                activation='relu',
                strides=1)(conv_1)
flat = Flatten()(conv_2) # Since we are passing flat data anyway, we probably don't need this.
hidden = Dense(intermediate_dim, activation='relu')(flat)
z_mean = Dense(latent_dim)(hidden)
z_log_var = Dense(latent_dim)(hidden)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var ) * epsilon # the original VAE divides z_log_var with two -- why?

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_var])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])



# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')

h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) # Double check wtf this is supposed to be
    return xent_loss + kl_loss

vae = Model(x, x_decoded_mean)
vae.compile(optimizer='adam', loss=vae_loss) # 'rmsprop'
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (100, 1, 784)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (100, 1, 1)          4705        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (100, 1, 64)         448         conv1d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (100, 64)            0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
dense_4 (D

In [11]:
from keras.datasets import mnist

img_rows, img_cols = 1,28*28
original_img_size = (img_rows, img_cols)

# train the VAE on MNIST digits
(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((x_train.shape[0],) + original_img_size)

print('x_train.shape:', x_train.shape)

N = 1000
epochs = 100
batch_size = int(N/10)
vae.fit(x_train[0:N,:], x_train[0:N,:],
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)

x_train.shape: (60000, 1, 784)
Epoch 1/100
1000/1000 [==============================] - 1s 1ms/step - loss: 1169.9780
Epoch 2/100
1000/1000 [==============================] - 1s 1ms/step - loss: 4747.1548
Epoch 3/100
1000/1000 [==============================] - 1s 1ms/step - loss: 9004.4316
Epoch 4/100
1000/1000 [==============================] - 1s 1ms/step - loss: 10038.3535
Epoch 5/100
1000/1000 [==============================] - 1s 1ms/step - loss: 7186.3599
Epoch 6/100
1000/1000 [==============================] - 1s 1ms/step - loss: 6557.7183
Epoch 7/100
1000/1000 [==============================] - 1s 1ms/step - loss: 6371.3677
Epoch 8/100
1000/1000 [==============================] - 1s 1ms/step - loss: 6293.7485
Epoch 9/100
1000/1000 [==============================] - 1s 1ms/step - loss: 6242.7671
Epoch 10/100
1000/1000 [==============================] - 1s 1ms/step - loss: 6246.0151
Epoch 11/100
1000/1000 [==============================] - 1s 1ms/step - loss: 6255.0381
Epoch 12/

1000/1000 [==============================] - 1s 911us/step - loss: 5902.9458
Epoch 94/100
1000/1000 [==============================] - 1s 990us/step - loss: 5900.6768
Epoch 95/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5896.8774
Epoch 96/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5895.3408
Epoch 97/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5893.9971
Epoch 98/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5890.9468
Epoch 99/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5888.3589
Epoch 100/100
1000/1000 [==============================] - 1s 1ms/step - loss: 5887.1504
